## ```torch.jit.script```

```python
torch.jit.script(obj, optimize=None, _frames_up=0, _rcb=None)
```

Scripting a function or ```nn.Module``` will inspect the source code, compile it as TorchScript code and return ```ScriptModule``` or ```ScriptFunction```.

### Parameters

<b>obj</b>(callable, class, or ```nn.Module```) - The ```nn.Module```, function or class type to be compiled.

### Returns

If ```obj``` is ```nn.Module```, ```script``` returns a ```ScriptModule``` object. If ```obj``` is a standalone Function, a ```ScriptFunction``` will be returned

## 1. Scripting a Function

The ```@torch.jit.script``` decorator will construct a ```ScriptFunction``` by compiling the body of the function 

In [1]:
import torch

@torch.jit.script
def foo(x, y):
    if x.max() > y.max():
        r = x
    else:
        r = y
    return r

print(type(foo)) # torch.jit.ScriptFunction

# Call the function using TorchScript interpreter
foo(torch.ones(2, 2), torch.ones(2, 2))

<class 'torch.jit.ScriptFunction'>


tensor([[1., 1.],
        [1., 1.]])

## 2. Scripting an ```nn.Module```

Scripting an ```nn.Module``` by default will compile the ```forward``` method and recursively compile any methods, submodules, and functions called by ```forward```.

If a ```nn.Module``` only uses features supported in TorchScript, no changes to the original module code should be necessary. ```script``` will construct ScriptModule that has copies of the attributes, parameters, and methods of the original module.

### Example (Scripting a Single Module)

```python
# Scripting a Single Module
import torch
import torch.nn as nn

class MyModule(nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        
        # This parameter will be copied to the new ScriptModule
        self.weight = nn.Parameter(torch.randn(N, M))

        # When the submodule is used, it will be compiled
        self.linear = nn.Linear(N, M)
    
    def forward(self, x):
        output = self.weight.mv(x)

        # This calls the `forward` method of `nn.Linear` module, which will
        # cause the `self.linear` submodule to be compiled to a `ScriptModule`
        output = self.linear(output)
        return output

scripted_module = torch.jit.script(MyModule(2, 3)) 
print(scripted_module)
```

In [3]:
%run Code/Scripting_nnModule/ScriptingSingleModule.py

RecursiveScriptModule(
  original_name=MyModule
  (linear): RecursiveScriptModule(original_name=Linear)
)


### Example (Scripting a Module with traced SubModules)

```python
# scripting a module with traced submodules
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()

        # torch.jit.trace produces the ScriptModule's conv1 and conv2
        self.conv1 = torch.jit.trace(nn.Conv2d(1, 10, 5), torch.random(1, 1, 16, 16))
        self.conv2 = torch.jit.trace(nn.Conv2d(20, 20, 5), torch.rand(1, 20, 16, 16))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

scripted_module = torch.jit.script(MyModule()) 
print(scripted_module)
```

In [5]:
%run Code/Scripting_nnModule/ScriptingModule_TracedSubmodules.py

RecursiveScriptModule(
  original_name=MyModule
  (conv1): Conv2d(original_name=Conv2d)
  (conv2): Conv2d(original_name=Conv2d)
)


To compile a method other than ```forward```, add the ```@torch.jit.export``` decorator to the method. 

To opt out of compilation use ```@torch.jit.unused``` or ```@torch.jit.ignore```

### Example (An Exported and ignored method in  a Module)

```python
# FileName :- Exported_IgnoredMethod.py

# An Exported and ignored method in a Module
import torch
import torch.nn as nn

class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()

    @torch.jit.export
    def some_entry_point(self, input):
        return input + 10
    
    @torch.jit.ignore
    def python_only_fn(self, input):
        # This Function won't be compiled
        import pdb
        pdb.set_trace()
    
    def forward(self, input):
        if self.training:
            self.python_only_fn(input)
        return input * 99
    
scripted_module = torch.jit.script(MyModule())
print(scripted_module.some_entry_point(torch.randn(2, 2)))
print(scripted_module(torch.randn(2, 2)))
```

In [6]:
%run Code/Scripting_nnModule/Exported_IgnoredMethod.py

tensor([[10.4527,  8.8221],
        [ 9.9924,  8.8424]])
--Return--
None
> d:\ml\pytorch\pytorch_manual\creating torchscript code\code\scripting_nnmodule\exported_ignoredmethod.py(17)python_only_fn()
     15         # This Function won't be compiled
     16         import pdb
---> 17         pdb.set_trace()
     18 
     19     def forward(self, input):

ipdb> 1
1
ipdb> 12
12
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
tensor([[ 99.2354, -38.8926],
        [ -6.6966,   1.1183]])
